## Sagemaker
pg no: 68

In [4]:
import os
os.listdir()

['.ipynb_checkpoints',
 '1. sage maker role working .ipynb',
 '1.Using the SageMaker Autopilot SDK.ipynb',
 'bank-additional-full.csv',
 'Boston_Housing.csv',
 'Final.ipynb',
 'Mnist Sagemaker.ipynb',
 'mnist2.0.py',
 'mnist2.0.txt',
 'mnist2.0.zip',
 'Only_Run_This.ipynb',
 'preprocessing (155).py',
 'preprocessing (47).py',
 'preprocessing.py',
 'preprocessing2.py',
 'SageMaker Autopilot (2).ipynb',
 'SageMaker Autopilot.ipynb',
 'steps_to_Create_a_rust_project.txt',
 'Untitled (2).ipynb',
 'Untitled (3).ipynb',
 'Untitled.ipynb',
 'untitled.py']

In [8]:
import pandas as pd
P = ('bank-additional-full.csv' sep=';')
data = pd.read_csv(P,sep = ';')
print(data.shape)
print()
display(data[:2])

SyntaxError: invalid syntax (<ipython-input-8-4574f37994ee>, line 2)

### Uploading datafile in current s3 Bucket
Now, let's upload the dataset to Amazon S3. We'll use a default bucket automatically created by SageMaker in the region we're running in. We'll just add a prefix to keep things nice and tidy:
    

In [1]:
import sagemaker
#P = 'Bank Additional/bank-additional/bank-additional-full.csv'
#prefix = 'sagemaker/DEMO-smprocessing/input'
#input_data = sagemaker.Session().upload_data(path=P, key_prefix=prefix)

### Writing a processing script with scikit-learn
Container paths are provided when we configure the job itself. Here's what we'll use:

• The input dataset: /opt/ml/processing/input

• The processed training set: /opt/ml/processing/train

• The processed test set: /opt/ml/processing/test

In [4]:
# just writing
# Will execute it later
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--train-test-split-ratio',type=float, default=0.3)
args, _ = parser.parse_known_args()
print('Received arguments {}'.format(args))
split_ratio = args.train_test_split_ratio

Received arguments Namespace(train_test_split_ratio=0.3)


In [5]:
import os
import pandas as pd # 425

In [6]:
#input_data_path = os.path.join('/opt/ml/processing/input', 'bank-additional-full.csv')

In [7]:
#input_data_path
input_data_path = P

In [8]:
df = pd.read_csv(input_data_path,sep = ';')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [9]:
# Then, we remove any line with missing values, as well as duplicate lines:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [10]:
#Then, we count negative and positive samples, and display the class ratio.
#This will tell us how unbalanced the dataset is :

one_class = df[df['y']=='yes']
one_class_count = one_class.shape[0]
zero_class = df[df['y']=='no']
zero_class_count = zero_class.shape[0]
zero_to_one_ratio = zero_class_count/one_class_count
print("Ratio: %.2f" % zero_to_one_ratio)

Ratio: 7.88


Looking at the dataset, we can see a column named pdays, telling us how long ago a customer has been contacted. Some lines have a 999 value, and that looks pretty suspicious: indeed, this is a placeholder value meaning that a customer has never been contacted. To help the model understand this assumption, let's add a new column stating it explicitly:

In [11]:
import numpy as np
df['no_previous_contact'] = np.where(df['pdays'] == 999, 1, 0)

In the job column, we can see three categories (student, retired, and unemployed) that should probably be grouped to indicate that these customers don't have a full-time job. Let's add another column:

In [12]:
df['not_working'] = np.where(np.in1d(df['job'], ['student', 'retired', 'unemployed']), 1, 0)

Now, let's split the dataset into training and test sets. Scikit-learn has a convenient API for this, and we set the split ratio according to a command-line argument passed to the script

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('y', axis=1),df['y'],
                                                    test_size=split_ratio, random_state=0)

The next step is to scale numerical features and to one-hot encode the categorical features. We'll use StandardScaler for the former, and OneHotEncoder for the latter

In [14]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [15]:
preprocess = make_column_transformer((['age', 'duration', 'campaign', 'pdays', 'previous'],
                                      StandardScaler()),(['job', 'marital', 'education', 'default', 'housing',
                                      'loan','contact', 'month', 'day_of_week','poutcome'],
                                       OneHotEncoder(sparse=False)))

Then, we process the training and test datasets:

In [16]:
#train_features = preprocess.fit_transform(X_train)

In [17]:
#test_features = preprocess.transform(X_test)

In [18]:
################# Giving eroor doing it other way ################################

In [19]:
df.head(1)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,no_previous_contact,not_working
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,1,0


In [20]:
#from sklearn.preprocessing import StandardScaler, OneHotEncoder
#columns_to_encode = ['age', 'duration', 'campaign', 'pdays', 'previous']
#columns_to_scale=['job', 'marital', 'education', 'default', 'housing','loan','contact', 'month', 'day_of_week','poutcome']

# Instantiate encoder/scaler
#scaler = StandardScaler()
#ohe    = OneHotEncoder(sparse=False)

# Scale and Encode Separate Columns
#scaled_columns  = scaler.fit_transform(df[columns_to_scale]) 
#encoded_columns = ohe.fit_transform(df[columns_to_encode])

# Concatenate (Column-Bind) Processed Columns Back Together
#processed_data = np.concatenate([scaled_columns, encoded_columns], axis=1)

#df # not working

In [21]:
#########################

In [22]:
df.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,no_previous_contact,not_working
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,1,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,1,0


In [24]:
# working
# Code link
# https://stackoverflow.com/questions/43798377/one-hot-encode-categorical-variables-and-scale-continuous-ones-simultaneouely

 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Instantiate encoder/scaler
scaler = StandardScaler()
ohe    = OneHotEncoder(sparse=False)

columns_to_scale = ['age', 'duration', 'campaign', 'pdays', 'previous']
columns_to_encode=['job', 'marital', 'education', 'default', 'housing','loan','contact', 'month', 'day_of_week','poutcome']



p = Pipeline(
    [("coltransformer", ColumnTransformer(
        transformers=[
            ("assessments", Pipeline([("scale", scaler)]), columns_to_scale),
            ("ranks", Pipeline([("encode", ohe)]), columns_to_encode),
        ]),
    )]
)

#print(pd.DataFrame(p.fit_transform(df)).head())
#new_df = pd.DataFrame(p.fit_transform(df))
#new_df.head(2)
train_features = pd.DataFrame(p.fit_transform(X_train))
test_features = pd.DataFrame(p.fit_transform(X_test))

In [ ]:
####################################

10. Finally, we save the processed datasets, separating the features and labels. They're saved to user-defined locations in the container, and SageMaker Processing will automatically copy the files to S3 before terminating the job:


In [41]:
# ???
#train_features_output_path = os.path.join('/opt/ml/processing/train','train_features.csv')
#train_labels_output_path = os.path.join('/opt/ml/processing/train','train_labels.csv')

train_features_output_path = '/opt/ml/processing/train/train_features.csv'
train_labels_output_path = '/opt/ml/processing/train/train_labels.csv'

train_features_output_path

'/opt/ml/processing/train/train_features.csv'

In [42]:
import os
os.listdir("Bank Additional/bank-additional")

['.DS_Store',
 '.Rhistory',
 'bank-additional-full.csv',
 'bank-additional-names.txt',
 'bank-additional.csv']

In [43]:
"""base_dir = 'cats_and_dogs_small' \n",
    "os.mkdir(base_dir)                          # created new empty folder in current directory\n",
    "train_dir = os.path.join(base_dir, 'train')\n",
    "os.mkdir(train_dir)                         # us folder ke ander train ka folder banaya\n",
    "validation_dir = os.path.join(base_dir, 'validation')\n",
    "os.mkdir(validation_dir)                    # and so on\n",
    "test_dir = os.path.join(base_dir, 'test')\n",
    "os.mkdir(test_dir)"
   ]"""

'base_dir = \'cats_and_dogs_small\' \n",\n    "os.mkdir(base_dir)                          # created new empty folder in current directory\n",\n    "train_dir = os.path.join(base_dir, \'train\')\n",\n    "os.mkdir(train_dir)                         # us folder ke ander train ka folder banaya\n",\n    "validation_dir = os.path.join(base_dir, \'validation\')\n",\n    "os.mkdir(validation_dir)                    # and so on\n",\n    "test_dir = os.path.join(base_dir, \'test\')\n",\n    "os.mkdir(test_dir)"\n   ]'

In [52]:
# ???
#pd.DataFrame(train_features).to_csv(train_features_output_path,header=False, index=False)
#pd.DataFrame(test_features).to_csv(test_features_output_path,header=False, index=False)

pd.DataFrame(train_features).to_csv('opt/ml/processing/train',header=False, index=False)
pd.DataFrame(test_features).to_csv('opt/ml/processing/train',header=False, index=False)

PermissionError: [Errno 13] Permission denied: 'opt/ml/processing/train'

In [49]:
os.listdir("opt/ml/processing/train")

[]